# 理论基础
## 1. 分词器概念
分词器（Tokenizer）是指将自然语言文本转换为模型可理解的向量。包含两个任务，第一个是将自然语言文本进行分词，转换为最小基本单元（单词、子词甚至是单个字符），第二个任务就是将自然语言文本映射为一系列的索引。
tokens：可以是单词、子词、甚至是单个字符
vocabulary：词表，token到整数索引的映射，通常按照首字母或者出现频率排序

## 2. 分词器有什么作用
1. 将自然语言文本转换为模型可以输入的序列
2. 构建词汇表，用有限个子词覆盖大部分文本
3. 处理不在词汇表里面的词（out of vacabulary,OOV问题）：将未遇到的词拆分为词汇表里面的子词。
4. 保持文本的语义一致性

## 3. 为什么要训练分词器
1. 为了得到更小的词汇表，这样可以降低嵌入矩阵的参数量，例如词汇表大小为1000，嵌入维度为512，那么嵌入矩阵的大小为（1000，512）
2. 提高模型的泛化性：不认识的词可以拆分成多个子词


# 代码实现

## 数据准备
一问一答

In [1]:
import json
from typing import Generator

def read_text_from_jsonl(file_path: str) -> Generator[str, None, None]:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # 字典的形式加载
            data = json.loads(line)
            # yield关键字用来生成一个生成器函数，每次调用next()函数，就执行一次yield
            yield data['text']

## 加载分词器

中文分词器库：Jieba、HanLP、THULAC
英文及多语言分词器库：NLTK、SpaCy

本项目使用Hugging Face的 `tokenizers` 库，侧重于为各类预训练模型服务，通用性更强，支持多种语言和多种分词算法（如 BPE、WordPiece 等），提供对分词模型的训练、词汇表管理等功能。

`tokenizers` 库结构和组件：
* Tokenizer：核心类，负责将文本转换为token
* Normalizer：文本预处理，如转换为小写等
* PreTokenizer：处理文本的初步分割
* Model：定义如何将文本切割成token，支持BPE、WordPiece等
* PostProcessor：处理tokens的最后步骤，如添加特殊标记


In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers

# 定义分词器
tokenizer = Tokenizer(models.BPE())  # 使用BPE分词模型
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)  # 使用字节级别的预分词器，不在文本开头添加空格

# 定义特殊Token
special_tokens = ["<unk>", "<s>", "/s"]

# 设置训练器
trainer = trainers.BpeTrainer(vocab_size=6400,    
    special_tokens=special_tokens,  # 确保这三个token被包含
    show_progress=True,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet())

## 训练分词器

In [ ]:
# 读取文本数据
texts = read_text_from_jsonl(data_path)
# 训练 tokenizer
tokenizer.train_from_iterator(texts, trainer=trainer)

## 保存分类器
设置解码器也为字节形式，保持一致。保存配置文件是为了方便复用和理解。

In [ ]:
# 设置解码器
tokenizer.decoder = decoders.ByteLevel()

# 保存tokenizer
tokenizer_dir = "/root/autodl-tmp/miniDeepSeek/model/miniDeepSeek_tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json"))
tokenizer.model.save("/root/autodl-tmp/miniDeepSeek/model/miniDeepSeek_tokenizer")

# 手动创建配置文件
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": True,
    "added_tokens_decoder": {
        "0": {
            "content": "<unk>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
            },
        "1": {
            "content": "<s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
            },
        "2": {
            "content": "</s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
            }
    },
    "bos_token": "<s>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "</s>",
    "legacy": True,
    "model_max_length": 1000000000000000019884624838656,
    "pad_token": None,
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<unk>",
    "use_default_system_prompt": False,
    "chat_template": "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<s>user\\n' + content + '</s>\\n<s>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '</s>' + '\\n' }}{% endif %}{% endfor %}"
}

# 保存配置文件
with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer 保存成功！")

## 评估分词器

In [ ]:
from transformers import AutoTokenizer

# 加载预训练的tokenizer
tokenizer = AutoTokenizer.from_pretrained("./model/miniDeepSeek_tokenizer")

# 测试一段对话
messages = [
    {"role": "system", "content": "你是一个优秀的聊天机器人，总是给我正确的回应！"},
    {"role": "user", "content": '是椭圆形的'},
    {"role": "assistant", "content": '456'},
    {"role": "user", "content": '456'},
    {"role": "assistant", "content": '789'}
]

# 使用模板进行文本处理
new_prompt = tokenizer.apply_chat_template(messages, tokenize=True)
print(new_prompt)